In [1]:
pip install pandas numpy scikit-learn


In [2]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
file_path = 'car_prediction_data.csv'
data = pd.read_csv(file_path)

print(data.columns)

# Show the first few rows to understand the data structure
print(data.head())

# Data preprocessing and transformation (example: dropping missing values)
data.dropna(inplace=True)

# Assuming the dataset has a target column named 'target' and features columns
features = data.drop('Selling_Price', axis=1)
target = data['Selling_Price']




Index(['Car_Name', 'Year', 'Selling_Price', 'Present_Price', 'Kms_Driven',
       'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner'],
      dtype='object')
  Car_Name  Year  Selling_Price  Present_Price  Kms_Driven Fuel_Type  \
0     ritz  2014           3.35           5.59       27000    Petrol   
1      sx4  2013           4.75           9.54       43000    Diesel   
2     ciaz  2017           7.25           9.85        6900    Petrol   
3  wagon r  2011           2.85           4.15        5200    Petrol   
4    swift  2014           4.60           6.87       42450    Diesel   

  Seller_Type Transmission  Owner  
0      Dealer       Manual      0  
1      Dealer       Manual      0  
2      Dealer       Manual      0  
3      Dealer       Manual      0  
4      Dealer       Manual      0  


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


In [5]:
# Select columns excluding 'Car_Name' (assuming it's a non-numeric column)
cols_to_use = ['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']

# Extract data with selected columns
data_selected = data[cols_to_use]

# Data preprocessing: Convert 'Year' to Age
data_selected['Age'] = 2024 - data_selected['Year']
data_selected.drop('Year', axis=1, inplace=True)

# Select categorical columns
categorical_cols = ['Fuel_Type', 'Seller_Type', 'Transmission']

# Perform one-hot encoding on categorical columns
data_encoded = pd.get_dummies(data_selected, columns=categorical_cols)

# Data enrichment: Create a feature based on car usage
data_encoded['Usage_Category'] = pd.cut(data_encoded['Kms_Driven'], bins=[0, 10000, 50000, 100000, float('inf')],
                                       labels=['Low', 'Moderate', 'High', 'Very High'])

# Perform one-hot encoding on the 'Usage_Category' column
data_encoded = pd.get_dummies(data_encoded, columns=['Usage_Category'])

# Drop 'Kms_Driven' after creating the usage category feature
data_encoded.drop('Kms_Driven', axis=1, inplace=True)

# Data preprocessing and transformation (example: dropping missing values)
data_encoded.dropna(inplace=True)

# Assuming the dataset has a target column named 'Selling_Price' and features columns after encoding
features_encoded = data_encoded.drop('Selling_Price', axis=1)
target_encoded = data_encoded['Selling_Price']

# Split the dataset into training and testing sets
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(
    features_encoded, target_encoded, test_size=0.2, random_state=42
)

# Initialize a machine learning model (Random Forest Regressor in this case for price prediction)
model = RandomForestRegressor()

# Train the model
model.fit(X_train_encoded, y_train_encoded)

# Make predictions
predictions = model.predict(X_test_encoded)

# Evaluate the model
mse = mean_squared_error(y_test_encoded, predictions)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 0.8416697665061453


In [6]:
print(data_encoded.head())

   Selling_Price  Present_Price  Owner  Age  Fuel_Type_CNG  Fuel_Type_Diesel  \
0           3.35           5.59      0   10              0                 0   
1           4.75           9.54      0   11              0                 1   
2           7.25           9.85      0    7              0                 0   
3           2.85           4.15      0   13              0                 0   
4           4.60           6.87      0   10              0                 1   

   Fuel_Type_Petrol  Seller_Type_Dealer  Seller_Type_Individual  \
0                 1                   1                       0   
1                 0                   1                       0   
2                 1                   1                       0   
3                 1                   1                       0   
4                 0                   1                       0   

   Transmission_Automatic  Transmission_Manual  Usage_Category_Low  \
0                       0                    1

In [7]:
summary_stats = data_encoded.describe()
summary_stats

,Selling_Price,Present_Price,Owner,Age,Fuel_Type_CNG,Fuel_Type_Diesel,Fuel_Type_Petrol,Seller_Type_Dealer,Seller_Type_Individual,Transmission_Automatic,Transmission_Manual,Usage_Category_Low,Usage_Category_Moderate,Usage_Category_High,Usage_Category_Very High
count,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000,301.000000
mean,4.661296,7.628472,0.043189,10.372093,0.006645,0.199336,0.794020,0.647841,0.352159,0.132890,0.867110,0.156146,0.624585,0.192691,0.026578
std,5.082812,8.644115,0.247915,2.891554,0.081378,0.400166,0.405089,0.478439,0.478439,0.340021,0.340021,0.363598,0.485036,0.395069,0.161115
min,0.100000,0.320000,0.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.900000,1.200000,0.000000,8.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,3.600000,6.400000,0.000000,10.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
75%,6.000000,9.900000,0.000000,12.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000
max,35.000000,92.600000,3.000000,21.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Save the modified DataFrame back to a CSV file
data.to_csv('modified_car_prediction_data.csv', index=False)  # Specify a filename for the modified dataset

In [9]:
#loading the dataset into Database


# Connect to an SQLite database (if it doesn't exist, it will be created)
conn = sqlite3.connect('etl.db')


data_encoded.to_sql('car', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

In [10]:
# Load the dataset
file_path = 'framingham.csv'
data1 = pd.read_csv(file_path)

# Check for missing values in the dataset
null_values = data1.isnull().sum()

print(null_values)


male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64


In [11]:
print(data1.columns)

Index(['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
       'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
       'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD'],
      dtype='object')


In [12]:
# Replace null values in numerical columns with the mean
data1.fillna(data1.mean(), inplace=True)




In [13]:
data1.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
TenYearCHD         0
dtype: int64

In [14]:
# Select columns excluding 'TenYearCHD'
cols_to_use = ['male', 'age', 'education', 'currentSmoker', 'cigsPerDay', 'BPMeds',
               'prevalentStroke', 'prevalentHyp', 'diabetes', 'totChol', 'sysBP',
               'diaBP', 'BMI', 'heartRate', 'glucose', 'TenYearCHD']

# Extract data with selected columns
data_selected = data1[cols_to_use]

# Handle missing values (assuming 'education' has missing values)
data_selected['education'].fillna(data_selected['education'].median(), inplace=True)
# Handle other columns with missing values as needed

# Create BMI Category
data_selected['BMI_Category'] = pd.cut(data_selected['BMI'], bins=[0, 18.5, 24.9, 29.9, float('inf')],
                                       labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

# Perform one-hot encoding on the 'BMI_Category' column
data_selected = pd.get_dummies(data_selected, columns=['BMI_Category'])

# Drop the original BMI column after creating the BMI category feature
data_selected.drop('BMI', axis=1, inplace=True)

# Assuming the dataset has a target column named 'TenYearCHD' and features columns after encoding
features_selected = data_selected.drop('TenYearCHD', axis=1)
target_selected = data_selected['TenYearCHD']

# Split the dataset into training and testing sets
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    features_selected, target_selected, test_size=0.2, random_state=42
)

# Initialize a machine learning model (Random Forest Classifier for classification)
model = RandomForestClassifier()

# Train the model
model.fit(X_train_selected, y_train_selected)

# Make predictions
predictions = model.predict(X_test_selected)

# Evaluate the model
accuracy = accuracy_score(y_test_selected, predictions)
print(f"Accuracy: {accuracy}")


Accuracy: 0.8584905660377359


In [15]:
# Assuming your DataFrame is named 'data'
summary_stats = data1.describe()
print(summary_stats)


              male          age    education  currentSmoker   cigsPerDay  \
count  4240.000000  4240.000000  4240.000000    4240.000000  4240.000000   
mean      0.429245    49.580189     1.979444       0.494104     9.005937   
std       0.495027     8.572942     1.007082       0.500024    11.881610   
min       0.000000    32.000000     1.000000       0.000000     0.000000   
25%       0.000000    42.000000     1.000000       0.000000     0.000000   
50%       0.000000    49.000000     2.000000       0.000000     0.000000   
75%       1.000000    56.000000     3.000000       1.000000    20.000000   
max       1.000000    70.000000     4.000000       1.000000    70.000000   

            BPMeds  prevalentStroke  prevalentHyp     diabetes      totChol  \
count  4240.000000      4240.000000   4240.000000  4240.000000  4240.000000   
mean      0.029615         0.005896      0.310613     0.025708   236.699523   
std       0.168481         0.076569      0.462799     0.158280    44.327521   

In [16]:
# Save the modified DataFrame back to a CSV file
data1.to_csv('modified_framingham.csv', index=False)  # Specify a filename for the modified dataset

In [17]:
#loading the dataset into Database

# Connect to an SQLite database (if it doesn't exist, it will be created)
conn = sqlite3.connect('etl.db')


data_selected.to_sql('health', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()

In [18]:
# Load the dataset
file_path = 'food-price-index-september-2023-seasonally-adjusted.csv'
data2 = pd.read_csv(file_path)

print(data2.columns)

Index(['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS',
       'Subject', 'Group', 'Series_title_1'],
      dtype='object')


In [19]:
data2.isnull().sum()

Series_reference    0
Period              0
Data_value          3
STATUS              0
UNITS               0
Subject             0
Group               0
Series_title_1      0
dtype: int64

In [20]:
# Fill null values in numerical columns with their mean
data2.fillna(data2.mean(), inplace=True)


<ipython-input-20-c6095afbdaf2>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data2.fillna(data2.mean(), inplace=True)


In [21]:
data2.isnull().sum()

Series_reference    0
Period              0
Data_value          0
STATUS              0
UNITS               0
Subject             0
Group               0
Series_title_1      0
dtype: int64

In [22]:
print(data2.columns)


Index(['Series_reference', 'Period', 'Data_value', 'STATUS', 'UNITS',
       'Subject', 'Group', 'Series_title_1'],
      dtype='object')


In [23]:
# Selecting column excluding 'Series_reference'(assuming they're non-numeric)
cols_to_use = ['Data_value','Period', 'STATUS', 'UNITS', 'Subject', 'Group', 'Series_title_1']

# Extract data with selected columns
data2_selected = data2[cols_to_use]

# Select categorical columns
categorical_cols = [ 'STATUS', 'UNITS', 'Subject', 'Group', 'Series_title_1']

# Perform one-hot encoding on categorical columns
data_encoded1 = pd.get_dummies(data2_selected, columns=categorical_cols)

# Data preprocessing and transformation (example: dropping missing values)
data_encoded1.dropna(inplace=True)

# Assuming the dataset has a target column named 'Data_value' and features columns after encoding
features_encoded = data_encoded1.drop('Data_value', axis=1)
target_encoded = data_encoded1['Data_value']

# Split the dataset into training and testing sets
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(
    features_encoded, target_encoded, test_size=0.2, random_state=42
)

# Initialize a machine learning model (Random Forest Regressor in this case)
model = RandomForestRegressor()

# Train the model
model.fit(X_train_encoded, y_train_encoded)

# Make predictions
predictions = model.predict(X_test_encoded)

# Evaluate the model
mse = mean_squared_error(y_test_encoded, predictions)
print(f"Mean Squared Error: {mse}")


Mean Squared Error: 4566.505978147908


In [24]:
print(data_encoded.columns)

Index(['Selling_Price', 'Present_Price', 'Owner', 'Age', 'Fuel_Type_CNG',
       'Fuel_Type_Diesel', 'Fuel_Type_Petrol', 'Seller_Type_Dealer',
       'Seller_Type_Individual', 'Transmission_Automatic',
       'Transmission_Manual', 'Usage_Category_Low', 'Usage_Category_Moderate',
       'Usage_Category_High', 'Usage_Category_Very High'],
      dtype='object')


In [25]:
# Assuming your DataFrame is named 'data'
summary_stats = data2.describe()
print(summary_stats)


            Period   Data_value
count  4244.000000  4244.000000
mean   2014.406140   963.753596
std       5.254114   126.821172
min    1999.060000   624.000000
25%    2010.060000   886.000000
50%    2014.110000   978.500000
75%    2019.040000  1030.000000
max    2023.090000  1376.000000


In [26]:
# Save the modified DataFrame back to a CSV file
data2.to_csv('modified_food_price.csv', index=False)  # Specify a filename for the modified dataset

In [28]:
#loading the dataset into Database

# Connect to an SQLite database (if it doesn't exist, it will be created)
conn = sqlite3.connect('etl.db')



data_encoded1.to_sql('food', conn, if_exists='replace', index=False)

# Commit changes and close the connection
conn.commit()
conn.close()